In [ ]:
import pandas as pd
import datetime
import glob
import os
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# read studied area
pts = pd.read_csv(r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\Points_locations\SEA_Points_Country.csv')
pts

## Weather

In [ ]:
# read weather data
weather = pd.read_csv(r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\weather.csv', index_col = 0)
weather

## Mobility

In [ ]:
# read facebook mobility data
path = r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\facebook_mobility'      
all_files = glob.glob(os.path.join(path, "*.csv"))     
facebook_mobility = pd.DataFrame(columns = [])
for f in all_files:
    df_from_each_file = pd.read_csv(f)
    facebook_mobility   = pd.concat([facebook_mobility, df_from_each_file], ignore_index=True)
facebook_mobility

In [ ]:
# select columns of interests only
facebook_mobility = facebook_mobility[['date', 'grid_lon', 'grid_lat', 'movement_value']]
facebook_mobility

In [ ]:
# rename to be consistent
facebook_mobility = facebook_mobility.rename(columns={'grid_lon': 'lon', 'grid_lat': 'lat'})
facebook_mobility

In [ ]:
facebook_mobility = facebook_mobility.groupby(['date', 'lon', 'lat']).agg({'movement_value':'sum'}).reset_index()
facebook_mobility

In [ ]:
# merge weather and facebook mobility, inner merge
merge_1 = pd.merge(weather, facebook_mobility, how = 'inner', on = ['date', 'lon', 'lat'])
merge_1

In [ ]:
# merge with pts to include country columns
merge_2 = pd.merge(merge_1, pts, how = 'inner', on = ['lon','lat'])
merge_2 = merge_2.dropna().sort_values(by = ['date', 'country', 'lon', 'lat'])
merge_2

In [ ]:
# change to datetime for further merge
merge_2['date'] =  pd.to_datetime(merge_2['date'], format='%Y-%m-%d')
merge_2

In [ ]:
# read apple mobility data
apple = pd.read_csv(r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\apple_mobility.csv', index_col = 0)
apple

In [ ]:
# change to datetime for further merge
apple['date'] =  pd.to_datetime(apple['date'], format='%Y-%m-%d')
apple

In [ ]:
# merge with apple mobility
merge_3 = pd.merge(merge_2, apple, how = 'inner', on = ['date','country'])
merge_3 = merge_3.dropna().sort_values(by = ['date', 'country', 'lon', 'lat'])
merge_3

## NO2

In [ ]:
# read and concat all daily NO2
path = r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\NO2_SEA_Land\NO2_raw_daily'      
all_files = glob.glob(os.path.join(path, "*.csv"))     
concatenated_NO2 = pd.DataFrame(columns = [])
for f in all_files:
    df_from_each_file = pd.read_csv(f)
    # create a new column to store year and month (extracted from file name)
    df_from_each_file['date'] = f[-14:-4]
    concatenated_NO2   = pd.concat([concatenated_NO2, df_from_each_file], ignore_index=True)
concatenated_NO2

In [ ]:
# drop useless column and rename column
concatenated_NO2 = concatenated_NO2.dropna().drop(columns = ['system:index']).rename(columns={'first': 'NO2'})
concatenated_NO2

In [ ]:
# change to datetime for further merge
concatenated_NO2['date'] =  pd.to_datetime(concatenated_NO2['date'], format='%Y-%m-%d')
concatenated_NO2

In [ ]:
# save to csv
# concatenated_NO2.to_csv(r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\NO2_SEA_Land\NO2_raw_daily\2020_2021_daily_NO2.csv', index = False)

In [ ]:
# merge with NO2
merge_4 = pd.merge(merge_3, concatenated_NO2, how = 'inner', on = ['date','lon', 'lat'])
merge_4 = merge_4.dropna().sort_values(by = ['date', 'country', 'lon', 'lat'])
merge_4

## Transboundary haze

In [ ]:
# read haze area
haze = pd.read_csv(r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\haze.csv')
haze

In [ ]:
# change to datetime for further merge
haze['date'] = pd.to_datetime(haze['date'], format = '%d/%m/%Y', errors='raise')
haze

In [ ]:
# get country code list
merge_4['Code'].unique()

In [ ]:
# change country code name in haze for further merge on code 
haze['Code'] = haze['Code'].replace(['KH', 'ID', 'MY', 'PH', 'SG', 'TH', 'VN'],['KHM', 'IDN', 'MYS', 'PHL', 'SGP', 'THA', 'VNM'])
haze

In [ ]:
# merge with NO2
merge_5 = pd.merge(merge_4, haze, how = 'inner', on = ['date','Code'])
merge_5 = merge_5.dropna().sort_values(by = ['date', 'lon', 'lat'])
merge_5

## Visualise

In [ ]:
#this is a public token
mapbox_token = 'pk.eyJ1IjoibGl5YW5neWFuZzUxNSIsImEiOiJjbDBuNmM3MjEwdGZjM2t0NHRqbmJidXFjIn0.8O9DnGkHPecl4jjk1ZqQUQ'

px.set_mapbox_access_token(mapbox_token)

In [ ]:
# merge_5 = pd.read_csv(r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\merged_data.csv', index_col = 0)
# merge_5

In [ ]:
# group by location, agg to mean
merge_5_by_loc = merge_5.groupby(['lon', 'lat','country']).agg({'NO2':'mean', 'driving':'mean','walking':'mean','movement_value':'mean'}).reset_index()
merge_5_by_loc

In [ ]:
# group by location, agg to max
merge_5_by_loc_max = merge_5.groupby(['lon', 'lat','country']).agg({'NO2':'max', 'driving':'max','walking':'max','movement_value':'max'}).reset_index()
merge_5_by_loc_max

In [ ]:
# plot to see NO2 in color and facebook mobility in size
fig_loc = px.scatter_mapbox(merge_5_by_loc, lat="lat", lon="lon", color = 'NO2', size = 'movement_value',
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=2)
fig_loc.show()

In [ ]:
# group by date, agg to mean
merge_5_by_date = merge_5.groupby(['date', 'country']).agg({'NO2':'mean', 'driving':'mean','walking':'mean','movement_value':'mean'}).reset_index()
merge_5_by_date

In [ ]:
# group by date, agg to max
merge_5_by_date_max = merge_5.groupby(['date', 'country']).agg({'NO2':'max', 'driving':'max','walking':'max','movement_value':'max'}).reset_index()
merge_5_by_date_max

In [ ]:
# plot the relationship between max NO2 and facebook mobility each day
fig = px.scatter(merge_5_by_date_max, x="movement_value", y="NO2", trendline="ols", color="country" )
#                  , facet_row="country")
fig.show()

In [ ]:
# plot the relationship between max NO2 and facebook mobility each day in log scale, can see a gap in between, indicating a possible lock down effet
fig = px.scatter(merge_5_by_date_max, x='movement_value', y='NO2', color='country', trendline='ols', log_x=True, log_y=True, hover_data=['date'])
fig.show()

In [ ]:
# plot the relationship between NO2 and facebook mobility for all, in log scale (can see some nosie at low movement, will filter them out in the MLP part)
fig = px.scatter(merge_5, x='movement_value', y='NO2', color='country', log_y=True,log_x=True, hover_data=['date','lon','lat'])
fig.show()

In [ ]:
merge_5['day'] = (merge_5['date'] - merge_5.iloc[0]['date']).dt.days
merge_5

In [ ]:
merge_5 = merge_5.rename(columns={'movement_value': 'facebook_movement', 'driving': 'apple_driving', 'driving': 'apple_driving', 'walking': 'apple_walking', 'tp': 'rainfall'})
merge_5

In [ ]:
# save to csv
merge_5.to_csv(r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\merged_data.csv')